In [1]:
%load_ext autoreload 
%autoreload 2

In [5]:
c.wandb_run_path

NameError: name 'c' is not defined

In [7]:
### Distribution ✨ ❇ Demo 💪 ### 
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '3'

### fancy logging variables, philosophically reminding us of the goal ###
fancy = dict(
		pe		= r'$V(X)',    				
		ke		= r'$\nabla^2',    		
		e		= r'$E',						
		log_psi	= r'$\log\psi', 			
		deltar	= r'$\delta_\mathrm{r}',	
		x		= r'$r_\mathrm{e}',
)

### pyfig ###
from pyfig import Pyfig
import numpy as np

arg = dict(
	charge = 0,
	spin  = 0,
	a = np.array([[0.0, 0.0, 0.0],]),
	a_z  = np.array([4.,]),
	n_b = 512, 
	n_sv = 32, 
	n_pv = 32, 
	n_corr = 20, 
	n_step = 10000, 
	log_metric_step = 50, 
	exp_name = 'junk',
	# sweep = {},
)

c = Pyfig(wandb_mode='disabled', arg=arg, submit=False, notebook=True)

n_device = c.n_device
print(f'🤖 {n_device} GPUs available')

from utils import run_cmds_server
cmd = c._run_terminal[['py', 'ipynb'][True]] + c.cmd
print(cmd)
server_out = run_cmds_server(c.server, c.user, cmd, c.server_project_path)[0]
print(server_out)

""" live plotting in another notebook """
""" copy lines and run in analysis while the exp is live """
# api = wandb.Api()
# run = api.run("<run-here>")
# c = run.config
# h = run.history()
# s = run.summary

FILE:  /home/amawi/projects/hwat/jax_version/pyfig.py
True False False False
🤖 1 GPUs available
jupyter nbconvert --to notebook --execute run.ipynb --project_root ~/projects --project hwat --data_dir ~/projects/data --run_path ~/projects/hwat/run.py --exp_name junk --exp_id Ms4tY81 --seed 808017424 --dtype float32 --n_step 10000 --log_metric_step 50 --log_state_step 10 --charge 0 --spin 0 --a [[0.0.0.]] --a_z [4.] --n_e 4 --n_u 2 --n_d 2 --n_b 512 --n_corr 20 --n_equil 10000 --acc_target 0.5 --with_sign False --n_sv 32 --n_pv 32 --n_fbv 160 --n_fb 2 --n_det 1 --terms_s_emb ['ra','ra_len'] --terms_p_emb ['rr','rr_len'] --job_type training --entity hwat --mail_type FAIL --partition sm3090 --nodes 1 --ntasks 8 --cpus_per_task 1 --time 0-12:00:00 --gres gpu:RTX3090:1 --output dump/tmp/o-%j.out --error dump/tmp/e-%j.err --job_name junk --TMP dump/tmp --project_path ~/projects/hwat --server_project_path ~/projects/hwat --n_device 1 --exp_path ~/projects/hwat/exp/junk/Ms4tY81 --server svol.fy

' copy lines and run in analysis while the exp is live '

In [8]:
c.cmd

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [9]:
### model (aka TrainState) ### 
from functools import partial
import jax
import optax
from flax.training.train_state import TrainState
from hwat import FermiNet

@partial(jax.pmap, axis_name='dev', in_axes=(0,0))
def create_train_state(rng, r):
	model = c.partial(FermiNet)
	params = model.init(rng, r)
	opt = optax.chain(optax.clip_by_block_rms(1.),optax.adamw(0.001))
	return TrainState.create(apply_fn=model.apply, params=params, tx=opt)


### train step ###
from jax import numpy as jnp
from hwat import compute_ke_b, compute_pe_b
from typing import NamedTuple

@partial(jax.pmap, in_axes=(0, 0))
def train_step(state, r_step):

	ke = compute_ke_b(state, r_step)
	pe = compute_pe_b(r_step, c.data.a, c.data.a_z)
	e = pe + ke
	
	e_mean_dist = jnp.mean(jnp.abs(jnp.median(e) - e))
	e_clip = jnp.clip(e, a_min=e-5*e_mean_dist, a_max=e+5*e_mean_dist)

	def loss(params):
		return ((e_clip - e_clip.mean())*state.apply_fn(params, r_step)).mean()
	
	grads = jax.grad(loss)(state.params)
	state = state.apply_gradients(grads=grads)
	
	v_tr = dict(
		params=state.params, grads=grads,
		e=e, pe=pe, ke=ke,
		r=r_step
	)

	return state, v_tr


### init variables ###
from utils import gen_rng
from hwat import init_r, get_center_points
from jax import random as rnd

rng, rng_p = gen_rng(rnd.PRNGKey(c.seed), c.n_device)
center_points = get_center_points(c.data.n_e, c.data.a)
r = init_r(rng_p, c.data.n_b, c.data.n_e, center_points, std=0.1)
deltar = jnp.array([0.02])[None, :].repeat(n_device, axis=0)

print(f"""exp/actual | 
	rng    : {(2,)}/{rng.shape} 
	rng_p  : {(c.n_device,2)}/{rng_p.shape} 
	cps    : {(c.data.n_e,3)}/{center_points.shape}
	r      : {(c.n_device, c.data.n_b, c.data.n_e, 3)}/{r.shape}
	deltar : {(c.n_device, 1)}/{deltar.shape}
""")


### init functions ### 
from hwat import sample_b

state = create_train_state(rng_p, r)
metro_hast = jax.pmap(partial(sample_b, n_corr=c.data.n_corr), in_axes=(0,0,0,0))


### train ###
import wandb
from hwat import keep_around_points
from utils import compute_metrix

wandb.define_metric("*", step_metric="tr/step")
for step in range(1, c.n_step+1):
	rng, rng_p = gen_rng(rng, c.n_device)

	r, acc, deltar = metro_hast(rng_p, state, r, deltar)
	r = keep_around_points(r, center_points, l=2.) if step < 1000 else r
	
	state, v_tr = train_step(state, r)

	if not (step % c.log_metric_step):
		metrix = compute_metrix(v_tr)
		wandb.log({'tr/step':step, **metrix})

exp/actual | 
	rng    : (2,)/(2,) 
	rng_p  : (1, 2)/(1, 2) 
	cps    : (4, 3)/(4, 3)
	r      : (1, 512, 4, 3)/(1, 512, 4, 3)
	deltar : (1, 1)/(1, 1)

path exists, leaving alone


In [ ]:
# ```{toggle} env vars and jax debug config notes
# ❇️ Magic & debug not currently used

# %load_ext autoreload
# %autoreload 2
# %env CUDA_VISIBLE_DEVICES='3'
# %env "WANDB_NOTEBOOK_NAME" "run.ipynb" # ❕same as notebook

# from jax.config import config
# config.update('jax_disable_jit', True)
# ```